# IMPORT FILES

In [1]:
#import files 
import pandas as pd
import numpy as np
from urllib import request
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import time

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("enable-automation")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument(" --disable-gpu")

driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)

[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:06<00:00, 1.13MB/s]
C:\Users\charl\AppData\Local\Temp\ipykernel_14012\2675240222.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)


# ACCESS TO INFORMATION ON FIRST WEBSITE

In this section we are going to scrap the data on the football observatory website

In [3]:
#first we want to access to https://football-observatory.com/
url = "https://football-observatory.com/IMG/sites/playerprofile/"
driver.get(url)

### Some initials tests

In [ ]:
### script convert perfomance to a dataframe ###

# /html/body/div/div/div/div[2]/div[2]
liste = driver.find_element('xpath','/html/body/div/div/div/div[2]/div[2]')

def convert_perf_to_frame(texte,name):
    liste = texte.split("\n")
    res = {}
    res["Name"]=name
    for i in range(4,len(liste),4):
        res[liste[i]+" (ratioToTeam)"] = liste[i+1]
        res[liste[i]+" (ratioToLeague)"] = liste[i+2]
    return res

dico = convert_perf_to_frame(liste.text,"Van Dijk")
df = pd.DataFrame.from_dict(dico, orient='index')
df

dico_test = {"c1":[0,2,3],"c2":[45,69,97]}
df_test = pd.DataFrame.from_dict(dico_test, orient='columns')
df_test

### Import transfermarkt's dataframe

In [4]:
data_joueur_transfermarkt = pd.read_csv("data_joueurs.csv")

In [44]:
#remove goalkeeper because not in CIES website
data_joueur_transfermarkt = data_joueur_transfermarkt[data_joueur_transfermarkt["poste"]!="Gardien"]

In [45]:
data_joueur_transfermarkt.head(5)

,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,valeur_marchande
3,Premier League,Manchester City,Rúben Dias,25,"1,87 m",Portugal,Défense - Défenseur central,29 sept. 2020,30 juin 2027,"75,00 mio. €"
4,Premier League,Manchester City,Aymeric Laporte,28,"1,89 m",Espagne,Défense - Défenseur central,30 janv. 2018,30 juin 2025,"38,00 mio. €"
5,Premier League,Manchester City,John Stones,28,"1,88 m",Angleterre,Défense - Défenseur central,9 août 2016,30 juin 2026,"30,00 mio. €"
6,Premier League,Manchester City,Nathan Aké,27,"1,80 m",Pays-Bas,Défense - Défenseur central,5 août 2020,30 juin 2025,"30,00 mio. €"
7,Premier League,Manchester City,Manuel Akanji,27,"1,88 m",Suisse,Défense - Défenseur central,1 sept. 2022,30 juin 2027,"30,00 mio. €"


In [46]:
#liste des joueurs à récupérer :
liste_joueurs = list(data_joueur_transfermarkt["nom_joueur"])

In [50]:
len(liste_joueurs)

2248

### Some tests

In [34]:
# //*[@id="player"]
search_box = driver.find_element('xpath','//*[@id="player"]')
search_box.clear()
search_box.send_keys("Rúben Dias")
auto_complete = driver.find_elements('xpath', '//*[@id="player"]')
auto_complete[0].click()

In [35]:
liste2 = driver.find_element('xpath','/html/body/div/div/div/div[2]/div[2]')

In [36]:
liste2.text

'Performance per game area\nRatio to team (×)\nRatio to league (×)\n1\nAir defence\n2.43\n2.60\n2\nGround defence\n2.38\n1.57\n3\nAir attack\n1.79\n1.13\n4\nDistribution\n1.30\n1.99\n5\nRecovery\n1.23\n1.15\n6\nChance creation\n0.50\n0.94\n7\nShooting\n0.23\n0.36\n8\nTake on\n0.11\n0.16'

In [ ]:
search_box = driver.find_element('xpath','//*[@id="player"]')
search_box.clear()
search_box.send_keys("Hugo")
auto_complete = driver.find_elements('xpath', '/html/body/div[2]/ul/li[1]')
auto_complete[0].click()

In [ ]:
liste3 = driver.find_element('xpath','/html/body/div/div/div/div[2]/div[2]')

In [ ]:
test = liste3.text.split("\n")
test

### Sript to scrap the data and create our final dataframe

In [64]:
#script pour récuperer les statistiques des joueurs
#listes noms / listes air defence ...
stats = []
liste_joueur_non_present = []
search_box = driver.find_element('xpath','//*[@id="player"]')
for player in liste_joueurs:
    search_box.clear()
    search_box.send_keys(player)
    time.sleep(1)
    auto_complete = driver.find_elements('xpath', '/html/body/div[2]/ul/li[1]')
    try:
        auto_complete[0].click()
        liste3 = driver.find_element('xpath','/html/body/div/div/div/div[2]/div[2]')
        liste3 = liste3.text.split("\n")
        stat = [liste3[5],liste3[6],liste3[9],liste3[10],liste3[13],liste3[14],liste3[17],liste3[18],liste3[21],liste3[22],liste3[25],liste3[26],liste3[29],liste3[30],liste3[33],liste3[34]]
        stats.append(stat)
    except:
        liste_joueur_non_present.append(player)


In [67]:
len(liste_joueur_non_present)

312

In [16]:
#uncomment to see the players not in the second website. If you make some research you will see some players are in jail or just never played so no stats
#liste_joueur_non_present

In [66]:
len(stats)

1936

In [71]:
#on va enlever les joueurs non présent dans le site football observatory
data_joueur_transfermarkt = data_joueur_transfermarkt[~data_joueur_transfermarkt.nom_joueur.isin(liste_joueur_non_present)]

In [78]:
#Take on
# Air defence
air_defense_ratio_to_team = [sublist[0] for sublist in stats]
data_joueur_transfermarkt["air_defense_ratio_to_team"] = air_defense_ratio_to_team
air_defense_ratio_to_league = [sublist[1] for sublist in stats]
data_joueur_transfermarkt["air_defense_ratio_to_league"] = air_defense_ratio_to_league
# Ground defence
ground_defence_ratio_to_team = [sublist[2] for sublist in stats]
data_joueur_transfermarkt["ground_defence_ratio_to_team"] = ground_defence_ratio_to_team
ground_defence_ratio_to_league = [sublist[3] for sublist in stats]
data_joueur_transfermarkt["ground_defence_ratio_to_league"] = ground_defence_ratio_to_league

# Air attack
air_attack_ratio_to_team = [sublist[4] for sublist in stats]
data_joueur_transfermarkt["air_attack_ratio_to_team"] = air_attack_ratio_to_team
air_attack_ratio_to_league = [sublist[5] for sublist in stats]
data_joueur_transfermarkt["air_attack_ratio_to_league"] = air_attack_ratio_to_league
# Distribution
distribution_ratio_to_team = [sublist[6] for sublist in stats]
data_joueur_transfermarkt["distribution_ratio_to_team"] = distribution_ratio_to_team
distribution_ratio_to_league = [sublist[7] for sublist in stats]
data_joueur_transfermarkt["distribution_ratio_to_league"] = distribution_ratio_to_league

# Recovery
recovery_ratio_to_team = [sublist[8] for sublist in stats]
data_joueur_transfermarkt["recovery_ratio_to_team"] = recovery_ratio_to_team
recovery_ratio_to_league = [sublist[9] for sublist in stats]
data_joueur_transfermarkt["recovery_ratio_to_league"] = recovery_ratio_to_league
# Chance creation
chance_creation_ratio_to_team = [sublist[10] for sublist in stats]
data_joueur_transfermarkt["chance_creation_ratio_to_team"] = chance_creation_ratio_to_team
chance_creation_ratio_to_league = [sublist[11] for sublist in stats]
data_joueur_transfermarkt["chance_creation_ratio_to_league"] = chance_creation_ratio_to_league

# Shooting
shooting_ratio_to_team = [sublist[12] for sublist in stats]
data_joueur_transfermarkt["shooting_ratio_to_team"] = shooting_ratio_to_team
shooting_ratio_to_league = [sublist[13] for sublist in stats]
data_joueur_transfermarkt["shooting_ratio_to_league"] = shooting_ratio_to_league
# Take on
take_on_ratio_to_team = [sublist[14] for sublist in stats]
data_joueur_transfermarkt["take_on_ratio_to_team"] = take_on_ratio_to_team
take_on_ratio_to_league = [sublist[15] for sublist in stats]
data_joueur_transfermarkt["take_on_ratio_to_league"] = take_on_ratio_to_league



C:\Users\charl\AppData\Local\Temp\ipykernel_960\3856589932.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joueur_transfermarkt["air_defense_ratio_to_team"] = air_defense_ratio_to_team
C:\Users\charl\AppData\Local\Temp\ipykernel_960\3856589932.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joueur_transfermarkt["air_defense_ratio_to_league"] = air_defense_ratio_to_league
C:\Users\charl\AppData\Local\Temp\ipykernel_960\3856589932.py:9: SettingWithCopyWarning: 
A value is trying to be set on

In [79]:
data_joueur_transfermarkt

,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,valeur_marchande,...,distribution_ratio_to_team,distribution_ratio_to_league,recovery_ratio_to_team,recovery_ratio_to_league,chance_creation_ratio_to_team,chance_creation_ratio_to_league,shooting_ratio_to_team,shooting_ratio_to_league,take_on_ratio_to_team,take_on_ratio_to_league
3,Premier League,Manchester City,Rúben Dias,25,"1,87 m",Portugal,Défense - Défenseur central,29 sept. 2020,30 juin 2027,"75,00 mio. €",...,1.30,1.99,1.23,1.15,0.50,0.94,0.23,0.36,0.11,0.16
4,Premier League,Manchester City,Aymeric Laporte,28,"1,89 m",Espagne,Défense - Défenseur central,30 janv. 2018,30 juin 2025,"38,00 mio. €",...,1.36,0.97,1.21,1.15,0.48,0.79,0.11,0.17,0.06,0.11
5,Premier League,Manchester City,John Stones,28,"1,88 m",Angleterre,Défense - Défenseur central,9 août 2016,30 juin 2026,"30,00 mio. €",...,1.16,0.86,1.01,0.94,0.50,0.92,0.39,0.58,0.26,0.41
6,Premier League,Manchester City,Nathan Aké,27,"1,80 m",Pays-Bas,Défense - Défenseur central,5 août 2020,30 juin 2025,"30,00 mio. €",...,1.14,0.79,0.74,0.56,0.24,0.39,0.13,0.17,0.09,0.20
7,Premier League,Manchester City,Manuel Akanji,27,"1,88 m",Suisse,Défense - Défenseur central,1 sept. 2022,30 juin 2027,"30,00 mio. €",...,1.14,1.11,0.57,0.53,0.51,0.85,0.33,0.48,0.17,0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538,Ligue 1,AC Ajaccio,Kevin Spadanuda,25,"1,77 m",Suisse,Attaquant - Ailier gauche,1 juil. 2022,30 juin 2025,800 K €,...,0.38,0.47,0.30,0.62,0.28,0.35,0.19,0.19,0.00,0.00
2540,Ligue 1,AC Ajaccio,Cyrille Bayala,26,"1,81 m",Burkina Faso,Attaquant - Ailier droit,15 janv. 2021,30 juin 2024,"1,00 mio. €",...,0.91,1.82,0.80,0.97,0.70,0.67,0.55,0.69,0.21,0.31
2542,Ligue 1,AC Ajaccio,Mounaïm El Idrissy,23,"1,81 m",France,Attaquant - Avant-centre,1 juil. 2019,30 juin 2023,"1,80 mio. €",...,0.53,0.50,0.52,0.56,0.47,0.59,0.28,0.33,0.26,0.38
2544,Ligue 1,AC Ajaccio,Romain Hamouma,35,"1,79 m",France,Attaquant - Avant-centre,1 juil. 2022,30 juin 2023,600 K €,...,1.09,1.99,0.80,0.72,0.51,0.56,0.30,0.36,0.00,0.00


In [90]:
data_joueur_transfermarkt[["nom_joueur","air_defense_ratio_to_league"]].sort_values("air_defense_ratio_to_league")

,nom_joueur,air_defense_ratio_to_league
2070,Vitinha,0.34
1112,Georginio Wijnaldum,0.36
1560,Noussair Mazraoui,0.72
2076,Pablo Sarabia,0.73
96,Marquinhos,0.74
...,...,...
2020,Sebastian Polter,8.74
2048,Philipp Hofmann,8.84
1908,Davie Selke,9.05
281,Dominic Calvert-Lewin,9.20


In [85]:
#data_joueur_transfermarkt[data_joueur_transfermarkt['fin_contrat'].isna()]

In [82]:
columns = data_joueur_transfermarkt.columns
for col in columns:
    res = (data_joueur_transfermarkt[col].isnull().sum() / len(data_joueur_transfermarkt)) * 100
    res = round(res,4)
    print(f'   - {col} : {res}%')

   - nom_championnat : 0.0%
   - nom_club : 0.0%
   - nom_joueur : 0.0%
   - age : 0.0%
   - taille : 0.0%
   - nationalite : 0.0%
   - poste : 0.0%
   - debut_equipe_actuel : 0.0%
   - fin_contrat : 1.8595%
   - valeur_marchande : 0.0%
   - air_defense_ratio_to_team : 0.0%
   - air_defense_ratio_to_league : 0.0%
   - ground_defence_ratio_to_team : 0.0%
   - ground_defence_ratio_to_league : 0.0%
   - air_attack_ratio_to_team : 0.0%
   - air_attack_ratio_to_league : 0.0%
   - distribution_ratio_to_team : 0.0%
   - distribution_ratio_to_league : 0.0%
   - recovery_ratio_to_team : 0.0%
   - recovery_ratio_to_league : 0.0%
   - chance_creation_ratio_to_team : 0.0%
   - chance_creation_ratio_to_league : 0.0%
   - shooting_ratio_to_team : 0.0%
   - shooting_ratio_to_league : 0.0%
   - take_on_ratio_to_team : 0.0%
   - take_on_ratio_to_league : 0.0%


### Create a csv of our final dataframe

In [80]:
data_joueur_transfermarkt.to_csv('data_joueurs_complet.csv')

# RECOMMENDATION SYSTEM

### Import dataframe and preprocessing

In [3]:
data_joueurs = pd.read_csv("data_joueurs_complet.csv")
data_joueurs.head(1)

,Unnamed: 0,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,...,distribution_ratio_to_team,distribution_ratio_to_league,recovery_ratio_to_team,recovery_ratio_to_league,chance_creation_ratio_to_team,chance_creation_ratio_to_league,shooting_ratio_to_team,shooting_ratio_to_league,take_on_ratio_to_team,take_on_ratio_to_league
0,3,Premier League,Manchester City,Rúben Dias,25,"1,87 m",Portugal,Défense - Défenseur central,29 sept. 2020,30 juin 2027,...,1.3,1.99,1.23,1.15,0.5,0.94,0.23,0.36,0.11,0.16


In [4]:
#one hot encoder sur le poste 
one_hot = pd.get_dummies(data_joueurs['poste'])
# Join the encoded df
data_joueurs = data_joueurs.join(one_hot)

In [5]:
data_joueurs.drop(columns='Unnamed: 0',inplace=True)

In [6]:
data_joueurs["taille"] = data_joueurs["taille"].str.replace("\xa0m", "")
data_joueurs["taille"] = data_joueurs["taille"].str.replace(",", ".")

In [7]:
def value_to_float(x):
    if 'K' in x:
        return float(x.replace(' K €', '')) * 0.001
    if 'mio.' in x:
        return float(x.replace(' mio. €', ''))



data_joueurs["valeur"] = data_joueurs["valeur_marchande"].str.replace(",", ".")
data_joueurs["valeur"] = data_joueurs["valeur"].apply(value_to_float)
data_joueurs["valeur"] = data_joueurs["valeur"].apply(pd.to_numeric)
column_to_move = data_joueurs.pop("valeur")
data_joueurs.insert(10, "valeur", column_to_move)

In [8]:
data_joueurs["age_normalized"] = MinMaxScaler().fit_transform(np.array(data_joueurs["age"]).reshape(-1,1))
data_joueurs["taille_normalized"] = MinMaxScaler().fit_transform(np.array(data_joueurs["taille"]).reshape(-1,1))
data_joueurs.head(1)

,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,valeur_marchande,...,Défense - Arrière droit,Défense - Arrière gauche,Défense - Défenseur central,Milieu de terrain - Milieu central,Milieu de terrain - Milieu droit,Milieu de terrain - Milieu défensif,Milieu de terrain - Milieu gauche,Milieu de terrain - Milieu offensif,age_normalized,taille_normalized
0,Premier League,Manchester City,Rúben Dias,25,1.87,Portugal,Défense - Défenseur central,29 sept. 2020,30 juin 2027,"75,00 mio. €",...,0,0,1,0,0,0,0,0,0.333333,0.55814


### Cosine similarity

In [9]:
X = data_joueurs.iloc[:, 11:].to_numpy()

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(X)

### SCRIPT DE TEST

In [12]:
def get_index_from_name(name):
    return data_joueurs[data_joueurs.nom_joueur == name].index.values[0]

def get_list_of_indexes(sorted_similar_players, nb_joueurs, valeur_max):
    i=1
    j=1
    liste_index_finale = []
    while(i<nb_joueurs+1):
        if data_joueurs.iloc[sorted_similar_players[j][0]].valeur<=valeur_max:
            liste_index_finale.append(sorted_similar_players[j][0])
            i+=1
        j+=1
    return liste_index_finale

def create_finale_df(sorted_similar_players, nb_joueurs, valeur_max):
    liste_index_finale = get_list_of_indexes(sorted_similar_players, nb_joueurs, valeur_max)
    res = data_joueurs.iloc[liste_index_finale]
    return res.iloc[:,:10]

def select_similar_players(cosine_matrix, player_name,nb_joueurs,valeur_max=300):
    player_index = get_index_from_name(player_name)
    similar_players = list(enumerate(cosine_matrix[player_index]))
    sorted_similar_players = sorted(similar_players, key=lambda x:x[1], reverse=True)
    return create_finale_df(sorted_similar_players, nb_joueurs, valeur_max)

pourquoi pas scrapper des infos mercatos pour tester notre modèle sur des rumeurs actuelles

#### Test 1 : Avec l'AS MONACO

In [119]:
#ici on va simuler des tests en temps que General Manager d'une équipe:
# Pour ce test on est à la tête de l'AS Monaco et on recherche un défenseur droit du profil d'Alessandro Florenzi sans contrainte de prix
df_similar = select_similar_players(cosine_sim,"Alessandro Florenzi",5)
df_similar

,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,valeur_marchande
461,LaLiga,Real Sociedad,Andoni Gorosabel,26,1.74,Espagne,Défense - Arrière droit,30 juin 2024,9 juin 2020,"10,00 mio. €"
382,Premier League,AFC Bournemouth,Adam Smith,31,1.74,Angleterre,Défense - Arrière droit,28 janv. 2014,30 juin 2024,"1,50 mio. €"
751,LaLiga,Real Valladolid,Luis Pérez,27,1.73,Espagne,Défense - Arrière droit,4 août 2020,30 juin 2025,"1,50 mio. €"
1426,Bundesliga,1.FSV Mainz 05,Silvan Widmer,29,1.83,Suisse,Défense - Arrière droit,9 juil. 2021,30 juin 2024,"5,00 mio. €"
558,LaLiga,Sevilla FC,Gonzalo Montiel,25,1.75,Argentine,Défense - Arrière droit,13 août 2021,30 juin 2026,"12,00 mio. €"


In [120]:
#pour 5 millions max 
df_similar = select_similar_players(cosine_sim,"Alessandro Florenzi",5,5)
df_similar

,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,valeur_marchande
382,Premier League,AFC Bournemouth,Adam Smith,31,1.74,Angleterre,Défense - Arrière droit,28 janv. 2014,30 juin 2024,"1,50 mio. €"
751,LaLiga,Real Valladolid,Luis Pérez,27,1.73,Espagne,Défense - Arrière droit,4 août 2020,30 juin 2025,"1,50 mio. €"
1426,Bundesliga,1.FSV Mainz 05,Silvan Widmer,29,1.83,Suisse,Défense - Arrière droit,9 juil. 2021,30 juin 2024,"5,00 mio. €"
597,LaLiga,CA Osasuna,Rubén Peña,31,1.70,Espagne,Défense - Arrière droit,30 juin 2025,,"2,50 mio. €"
1488,Bundesliga,1. FC Köln,Benno Schmitz,28,1.82,Allemagne,Défense - Arrière droit,1 juil. 2018,30 juin 2024,"3,00 mio. €"


#### Test 2 : Avec l'Olympique Lyonnais

Lyon manque d'un vrai bon défenseur central avec de l'expérience. Le directeur sportif dispose d'une enveloppe de 10 millions pour le mercato.

In [13]:
#Il observe d'abords si les joueurs comparés à Thiago Silva sont intéressant
df_similar = select_similar_players(cosine_sim,"Thiago Silva",10)
df_similar

,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,valeur_marchande
1664,Ligue 1,OGC Nice,Dante,39,1.88,Brésil,Défense - Défenseur central,23 août 2016,30 juin 2023,700 K €
1015,Serie A,FC Bologna,Kevin Bonifazi,26,1.87,Italie,Défense - Défenseur central,2 juil. 2021,30 juin 2025,"3,30 mio. €"
693,LaLiga,UD Almería,Chumi,23,1.85,Espagne,Défense - Défenseur central,10 sept. 2020,30 juin 2025,"2,00 mio. €"
1382,Bundesliga,VfB Stuttgart,Dan-Axel Zagadou,23,1.96,France,Défense - Défenseur central,30 juin 2026,NaN,"5,00 mio. €"
475,LaLiga,Villarreal CF,Pau Torres,25,1.91,Espagne,Défense - Défenseur central,1 juil. 2018,30 juin 2024,"50,00 mio. €"
204,Premier League,FC Everton,Yerry Mina,28,1.95,Colombie,Défense - Défenseur central,9 août 2018,30 juin 2023,"18,00 mio. €"
1682,Ligue 1,LOSC Lille,Tiago Djaló,22,1.90,Portugal,Défense - Défenseur central,1 août 2019,30 juin 2024,"12,00 mio. €"
477,LaLiga,Villarreal CF,Aïssa Mandi,31,1.84,Algérie,Défense - Défenseur central,1 juil. 2021,30 juin 2025,"4,00 mio. €"
4,Premier League,Manchester City,Manuel Akanji,27,1.88,Suisse,Défense - Défenseur central,1 sept. 2022,30 juin 2027,"30,00 mio. €"
729,LaLiga,Cádiz CF,Luis Hernández,33,1.82,Espagne,Défense - Défenseur central,31 janv. 2022,30 juin 2025,"1,00 mio. €"


In [123]:
#Maintenant il arrête de rêver et rentre son budget
df_similar = select_similar_players(cosine_sim,"Thiago Silva",10,10)
df_similar

,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,valeur_marchande
1664,Ligue 1,OGC Nice,Dante,39,1.88,Brésil,Défense - Défenseur central,23 août 2016,30 juin 2023,700 K €
1015,Serie A,FC Bologna,Kevin Bonifazi,26,1.87,Italie,Défense - Défenseur central,2 juil. 2021,30 juin 2025,"3,30 mio. €"
693,LaLiga,UD Almería,Chumi,23,1.85,Espagne,Défense - Défenseur central,10 sept. 2020,30 juin 2025,"2,00 mio. €"
1382,Bundesliga,VfB Stuttgart,Dan-Axel Zagadou,23,1.96,France,Défense - Défenseur central,30 juin 2026,NaN,"5,00 mio. €"
477,LaLiga,Villarreal CF,Aïssa Mandi,31,1.84,Algérie,Défense - Défenseur central,1 juil. 2021,30 juin 2025,"4,00 mio. €"
729,LaLiga,Cádiz CF,Luis Hernández,33,1.82,Espagne,Défense - Défenseur central,31 janv. 2022,30 juin 2025,"1,00 mio. €"
769,LaLiga,Elche CF,Pedro Bigas,32,1.81,Espagne,Défense - Défenseur central,30 juin 2023,NaN,"1,00 mio. €"
515,LaLiga,FC Valencia,Gabriel Paulista,32,1.87,Brésil,Défense - Défenseur central,18 août 2017,30 juin 2024,"4,00 mio. €"
1381,Bundesliga,VfB Stuttgart,Hiroki Ito,23,1.88,Japon,Défense - Défenseur central,1 juil. 2022,30 juin 2025,"5,50 mio. €"
994,Serie A,Udinese Calcio,Nehuén Pérez,22,1.84,Argentine,Défense - Défenseur central,29 juil. 2022,30 juin 2027,"8,00 mio. €"


In [21]:
df_similar = select_similar_players(cosine_sim,"Phil Foden",10,1)
df_similar

,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,valeur_marchande
1316,Bundesliga,Borussia Mönchengladbach,Rocco Reitz,20,1.76,Allemagne,Milieu de terrain - Milieu central,1 juil. 2020,30 juin 2024,650 K €
1161,Serie A,US Lecce,John Björkengren,24,1.80,Suède,Milieu de terrain - Milieu central,4 oct. 2020,30 juin 2023,600 K €
1515,Bundesliga,SV Werder Bremen,Nicolai Rapp,26,1.86,Allemagne,Milieu de terrain - Milieu central,1 juil. 2021,30 juin 2024,800 K €
1535,Bundesliga,FC Schalke 04,Danny Latza,33,1.79,Allemagne,Milieu de terrain - Milieu central,1 juil. 2021,30 juin 2024,600 K €
759,LaLiga,Real Valladolid,Anuar,27,1.73,Maroc,Milieu de terrain - Milieu central,1 juil. 2017,30 juin 2023,"1,00 mio. €"
231,Premier League,Wolverhampton Wanderers,Connor Ronan,24,1.71,Irlande,Milieu de terrain - Milieu central,1 janv. 2017,30 juin 2024,800 K €
722,LaLiga,RCD Mallorca,Lago Júnior,31,1.80,Côte d'Ivoire,Attaquant - Ailier gauche,27 janv. 2016,30 juin 2023,900 K €
1411,Bundesliga,1.FC Union Berlin,Kevin Möhwald,29,1.82,Allemagne,Milieu de terrain - Milieu central,30 août 2021,-,"1,00 mio. €"
1840,Ligue 1,ESTAC Troyes,Tristan Dingomé,31,1.74,Cameroun,Milieu de terrain - Milieu central,5 oct. 2020,30 juin 2023,600 K €
1116,Serie A,Hellas Verona,Filippo Terracciano,19,1.81,Italie,Milieu de terrain - Milieu central,1 juil. 2022,30 juin 2026,"1,00 mio. €"


In [21]:
df_similar = select_similar_players(cosine_sim,"Erling Haaland",3)
df_similar

,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,valeur_marchande
1622,Ligue 1,Stade Rennais FC,Matthis Abline,19,1.82,France,Attaquant - Avant-centre,1 juil. 2021,30 juin 2025,"3,00 mio. €"
828,Serie A,Inter Milan,Lautaro Martínez,25,1.74,Argentine,Attaquant - Avant-centre,4 juil. 2018,30 juin 2026,"75,00 mio. €"
1883,Ligue 1,Toulouse FC,Ado Onaiwu,27,1.80,Japon,Attaquant - Avant-centre,20 juil. 2021,30 juin 2024,"1,50 mio. €"
